In [16]:
import numpy as np
import pandas as pd
import pylab as pl
import geopandas as gpd
import os
import io
import requests
import geopandas as gpd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime
from matplotlib import gridspec
from fiona.crs import from_epsg
import shapely
import seaborn as sns
from sodapy import Socrata

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_row', 500)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


Collect data and filter the columns

In [2]:
url = 'https://data.cityofnewyork.us/resource/qiz3-axqb.csv'
s = requests.get(url).content
accidents = pd.read_csv(io.StringIO(s.decode('utf-8')))

In [3]:
accidents.head()

,:@computed_region_92fq_4b7q,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_sbqj_enih,:@computed_region_yeji_bk3q,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,date,latitude,location,location_address,location_city,location_state,location_zip,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,time,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,38.0,18182.0,36.0,57.0,2.0,BROOKLYN,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,2019-05-06T00:00:00.000,40.727470,POINT (-73.93623 40.72747),NaN,NaN,NaN,NaN,-73.93623,0,0,0,0,0,0,0,0,VARICK STREET,BRIDGEWATER STREET,17:50,4127839,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,NaN,11222.0
1,30.0,15310.0,54.0,62.0,3.0,QUEENS,Turning Improperly,Unspecified,NaN,NaN,NaN,NaN,2019-05-06T00:00:00.000,40.699660,POINT (-73.8988 40.69966),NaN,NaN,NaN,NaN,-73.89880,0,0,0,0,0,0,0,0,NORMAN STREET,FOREST AVENUE,10:45,4127805,Carry All,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,11385.0
2,50.0,11724.0,70.0,5.0,4.0,MANHATTAN,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,2019-05-06T00:00:00.000,40.729190,POINT (-73.98724 40.72919),NaN,NaN,NaN,NaN,-73.98724,0,0,0,0,0,0,0,0,2 AVENUE,EAST 9 STREET,8:41,4127103,Sedan,Pick-up Truck,NaN,NaN,NaN,10003.0
3,17.0,17614.0,61.0,40.0,2.0,BROOKLYN,Unspecified,Unspecified,NaN,NaN,NaN,1143 CLARKSON AVENUE,2019-05-06T00:00:00.000,40.660880,POINT (-73.92051 40.66088),NaN,NaN,NaN,NaN,-73.92051,0,0,0,0,0,0,0,0,NaN,NaN,9:40,4126728,Station Wagon/Sport Utility Vehicle,Box Truck,NaN,NaN,NaN,11212.0
4,40.0,11605.0,24.0,34.0,5.0,BRONX,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,2019-05-06T00:00:00.000,40.876495,POINT (-73.88321 40.876495),NaN,NaN,NaN,NaN,-73.88321,0,0,0,0,0,0,0,0,VANCORTLANDT AVENUE EAST,MOSHOLU PARKWAY,16:34,4127084,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,10458.0


In [4]:
accidents.shape

(1000, 38)

In [17]:
def getaAccidentdata(key, start_time, end_time):
    client = Socrata("data.cityofnewyork.us", key)
    results = client.get("qiz3-axqb", where = "date BETWEEN " + start_time + " AND " +  end_time, limit = 1000000000)
    
    return(pd.DataFrame.from_records(results))

In [18]:
accidents = getaAccidentdata('jjCh6OTqr3VrUwNTNl4bPGwAK',  "'2018-01-01T00:00:00'", "'2018-12-31T00:00:00'")

In [19]:
accidents.shape

(231467, 29)

In [20]:
accidents.to_csv("accidents_raw.csv", index = False)

In [8]:
accidents.head()

,borough,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,cross_street_name,date,latitude,location,longitude,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_persons_injured,number_of_persons_killed,off_street_name,on_street_name,time,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,zip_code
0,NaN,Passing or Lane Usage Improper,Unspecified,NaN,NaN,NaN,NaN,2018-06-29T00:00:00.000,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,HARLEM RIVER DRIVE,10:10,4026115,Sedan,NaN,NaN,NaN,NaN,NaN
1,NaN,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,NaN,2018-09-08T00:00:00.000,NaN,NaN,NaN,0,0,0,1,0,0,0,0,NaN,WILLIAMSBURG BRIDGE OUTER ROADWA,22:49,4026317,Sedan,NaN,NaN,NaN,NaN,NaN
2,NaN,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,2018-06-13T00:00:00.000,40.70089,"{u'type': u'Point', u'coordinates': [-73.94221...",-73.94221,0,0,0,0,0,0,0,0,NaN,BROADWAY,1:30,3919551,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN,NaN
3,BROOKLYN,Driver Inattention/Distraction,NaN,NaN,NaN,NaN,700 COLUMBIA STREET,2018-11-29T00:00:00.000,40.66947,"{u'type': u'Point', u'coordinates': [-74.00958...",-74.00958,0,0,0,0,0,0,0,0,NaN,NaN,10:45,4038301,Van,NaN,NaN,NaN,NaN,11231
4,MANHATTAN,Unspecified,NaN,NaN,NaN,NaN,520 first Avenue,2018-10-22T00:00:00.000,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN,14:00,4026463,Sedan,NaN,NaN,NaN,NaN,10016


In [9]:
sum(accidents.isna())

borough                           82302
contributing_factor_vehicle_1       694
contributing_factor_vehicle_2     34830
contributing_factor_vehicle_3    216771
contributing_factor_vehicle_4    228423
contributing_factor_vehicle_5    230656
cross_street_name                176538
date                                  0
latitude                          15114
location                          15114
longitude                         15114
number_of_cyclist_injured             0
number_of_cyclist_killed              0
number_of_motorist_injured            0
number_of_motorist_killed             0
number_of_pedestrians_injured         0
number_of_pedestrians_killed          0
number_of_persons_injured             5
number_of_persons_killed             14
off_street_name                  116551
on_street_name                    55642
time                                  0
unique_key                            0
vehicle_type_code1                 1396
vehicle_type_code2                49014


In [10]:
accidents = accidents[-accidents['latitude'].isna()]

In [11]:
accidents = accidents[['date', 'latitude', 'longitude']]

In [12]:
accidents.shape

(216353, 3)

In [13]:
sum(accidents.isna())

date         0
latitude     0
longitude    0
dtype: int64

In [14]:
accidents.head()

,date,latitude,longitude
2,2018-06-13T00:00:00.000,40.70089,-73.94221
3,2018-11-29T00:00:00.000,40.66947,-74.00958
13,2018-12-02T00:00:00.000,40.663605,-73.934395
14,2018-11-16T00:00:00.000,40.665985,-73.945305
15,2018-11-12T00:00:00.000,40.60968,-73.99298


In [15]:
accidents.to_csv("accidents_latlon.csv", index = False)

join with location

In [26]:
taxiZone = gpd.GeoDataFrame.from_file("taxi_zones/taxi_zones.shp")

In [27]:
taxiZone.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"(POLYGON ((1033269.243591294 172126.0078125, 1..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ..."


In [34]:
taxiZone_ft = taxiZone.to_crs(epsg = 2263)

In [32]:
accidents['lonlat'] = list(zip(accidents.longitude.astype(float),accidents.latitude.astype(float)))
accidents['geometry'] = accidents[['lonlat']].applymap(lambda x:shapely.geometry.Point(x))

accidents = gpd.GeoDataFrame(accidents)

accidents.crs = from_epsg(4326)

In [35]:
accidents_ft = accidents.to_crs(epsg=2263)

In [51]:
accidentsLocationId = gpd.sjoin(taxiZone_ft, accidents_ft)

In [52]:
accidentsLocationId = accidentsLocationId[['LocationID', 'date']].reset_index(drop = True)

In [53]:
accidentsLocationId.head()

,LocationID,date
0,2,2018-08-25T00:00:00.000
1,2,2018-05-04T00:00:00.000
2,2,2018-05-22T00:00:00.000
3,2,2018-05-11T00:00:00.000
4,2,2018-08-04T00:00:00.000


add day_id 

In [54]:
accidentsLocationId['date'] = pd.to_datetime(accidentsLocationId['date'])

In [55]:
from datetime import datetime
accidentsLocationId['day_id'] = (accidentsLocationId['date']).apply(lambda x: (x - datetime.strptime('2018-01-01',  "%Y-%M-%d")).days).tolist()
#accidentsLocationId['hour_id'] = (accidentsLocationId['date']).apply(lambda x: ((x - datetime.strptime('2018-01-01',  "%Y-%M-%d")).days)*24 + (x - datetime.strptime('2018-01-01',  "%Y-%M-%d")).seconds/3600).tolist()

In [56]:
accidentsLocationId.head()

,LocationID,date,day_id
0,2,2018-08-25,235
1,2,2018-05-04,122
2,2,2018-05-22,140
3,2,2018-05-11,129
4,2,2018-08-04,214


In [57]:
accidentsLocationId.to_csv('accident_day_clean.csv', index=False)

Total number of accidents in each loaction id in 2018

In [46]:
locationAccidentNumber = gpd.sjoin(taxiZone_ft, accidents_ft).groupby('LocationID').size().to_frame().reset_index().rename(columns = {0: 'Counts'})

In [47]:
locationAccidentNumber.head()

,LocationID,Counts
0,2,8
1,3,598
2,4,273
3,5,189
4,6,439


In [58]:
locationAccidentNumber.to_csv('accident_count_clean.csv', index=False)